## Error Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
rule_based_preds = pd.read_json('rule_test_pred.json')
svm_preds = pd.read_json('svm_test_pred.json')

ValueError: Expected object or value

In [ ]:
rule_based_preds.head()


In [ ]:
svm_reds.head()